# **Evaluating Your Options for Numerical Computing in Pure Python with Numba**

## **Prerequisites**

This tutorial assume proficiency in Python and the following libraries:

* NumPy
* Scikit-Learn
* Numba

Demo System - Benchmarking was performed on a T4 16 GB.

## **Problem Overview**

In this notebook, we  survey techniques for numerical computing in pure Python. We leverage a proxy geospatial nearest neighbor problem to guide us through an evaluation of several libraries and methodologies. In this use case, we aim to resolve geospatial observations to their nearest reference points with an added complexity. Our complication adds dynamics to the problem allowing each reference point to move and the set of observations to change on a reoccurring basis. These complexities imply a need to recompute each nearest neighbor at each timestep -- emphasizing the need for high performance techiques. 

Because of its simplicity and arithmetic intensity, we focus our attention on the brute force nearest neighbor technique using the haversine great circle distance formula as our distance metric. This is a popular formula used to calculate the distance between two points on earth.

<center><a href="https://en.wikipedia.org/wiki/Haversine_formula"><img src="./media/haversine-graphic.png" alt="Haversine" style="width: 150;"></a></center></br>

The graphic below illustrates the dynamic nature of our problem. From left to write, we can observe the dynamics of the system at each timestep -- with colored regions representing nearest neighbor decision boundaries for each reference point and points representing observations.

<center><img src="./media/visualization.PNG" alt="Visualization" style="width: 1000;"/></center>

In this notebook, we will evalutate the following single threaded CPU techniques:

* Numba CPU Kernel

In addition, we will evalutate several single GPU techniques:

* Hand tuned Numba CUDA Kernel

In the end, we'll compare their performance on a moderate sized problem (defined below) and expand our numerical computing toolbox with a few new tricks.

**Spoiler Alert -- The GPU techniques each out perform the CPU techniques by at least several orders of magnitude.**

Because many of the CPU functions take so long, we use the ```%%time``` magic function and comment out ```%%timeit``` to generate benchmarks.

Since many of the CPU techniques will take a very long time to complete, we provide an overview of the expected performance measured on a T4.

<center><img src="./media/small-problem-single.PNG" alt="PerfTable" style="width: 1250;"/></center>

# **Numba CPU/GPU Experiments**

In [ ]:
import cupy as cp

from numba import (cuda, 
                   uint32, 
                   int32, 
                   float32,
                   types, 
                   jit, 
                   prange)

from cupyx.time import repeat
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors
import rmm
from src.simulator import generate_geos

from src.utils import (query_available_memory, 
                       check_accuracy,
                       check_accuracy_h2h)

from math import sin, cos, sqrt, asin
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

Define constants for the size of our experiment and evaluation criteria.

In [ ]:
N_OBS, N_REF = 2**16, 2**13
N_OBS_VAL, N_REF_VAL = 500, 200 # check accuracy
print("Problem Size (N_OBS * N_REF): {:.2f}B".format(N_OBS * N_REF * 1e-9))

## **RAPIDS Memory Manager**

The RAPIDS Memory Manager (RMM) provides us finer grained control over memory allocations. Each memory allocation strategy comes with performance and data movement considerations.

Since our experiments will use a fair bit of GPU memory and we desire the fastest performance, let's reinitialize the RMM by pre-allocating our initial pool size on the device. With our available GPUs we will be allocating 14GB.

Note: be sure to free up memory from other notebooks/applications to maximize resources for these examples.

In [ ]:
device = cp.cuda.device.get_device_id()
rmm.rmm.reinitialize(pool_allocator=True, initial_pool_size=14000000000, devices=device)
cp.cuda.Device(device).use()

## **Generate Synthetic Dataset**

Generate a moderate sized synthetic dataset and smaller validation dataset to run our experiments using an included utility function. These datasets represent the following:

* ```d_obs``` contains ```N_OBS``` geospatial observations in radians on the GPU, used for our moderate scale benchmark
* ```d_ref``` contains ```N_REF``` geospatial reference points in radians on the GPU, used for our moderate scale benchmark
* ```d_obs_val``` contains ```N_OBS_VAL``` geospatial observations in radians on the GPU, used to validate accuracy
* ```d_ref_val``` contains ```N_REF_VAL``` geospatial reference points in radians on the GPU, used to validate accuracy
* ```h_``` prefixes represent copies of these data on the host to use with the CPU techniques

In [ ]:
d_ref = generate_geos(N_REF, random_state=1)
d_obs = generate_geos(N_OBS, random_state=2)

h_ref = d_ref.get()
h_obs = d_obs.get()

d_ref_val = generate_geos(N_REF_VAL, random_state=1)
d_obs_val = generate_geos(N_OBS_VAL, random_state=2)

h_ref_val = d_ref_val.get()
h_obs_val = d_obs_val.get()

## **Numba CPU Kernel**

[Numba](https://numba.pydata.org/) translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

Here, we leverage a Numba JIT compiled kernel based on our double for loop implementation and observe non-trivial speedups when compared to our baseline.

In [ ]:
@jit(nopython=True, fastmath=True)
def numba_cpu_haversine(lat1, lon1, lat2, lon2):

    return 2.0 * asin(
        sqrt(sin((lat2 - lat1) / 2.0)**2 + \
             cos(lat1) * \
             cos(lat2) * \
             sin((lon2 - lon1) / 2.0)**2)
    )      

@jit(nopython=True)
def numba_cpu_solve(a, b):
    
    out_idx = np.empty(
        (a.shape[0]), dtype=np.uint32)
    
    out_dist = np.empty(
        (a.shape[0]), dtype=np.float32)
    
    for obs_idx in range(a.shape[0]):
        
        glob_min_dist = 1e11
        glob_min_idx = 0
        
        for ref_idx in range(b.shape[0]):
            
            temp_dist = numba_cpu_haversine(
                a[obs_idx,0],
                a[obs_idx, 1],
                b[ref_idx, 0],
                b[ref_idx, 1])
            
            if temp_dist < glob_min_dist:
                glob_min_dist = temp_dist
                glob_min_idx = ref_idx
        
        out_dist[obs_idx] = glob_min_dist
        out_idx[obs_idx] = glob_min_idx
        
    return out_idx, out_dist

Verify our Numba CPU kernel is producing the correct results.

In [ ]:
out_idx_nb_cpu_val, out_dist_nb_cpu_val = \
    numba_cpu_solve(h_obs_val, h_ref_val)

print("Accuracy - Numba Single Threaded CPU:", 
      check_accuracy(
          d_obs_val, d_ref_val,
          out_idx_nb_cpu_val, out_dist_nb_cpu_val)
     )

This Numba CPU routine completes on the demo system in ~35.4s -- 9,220 x faster than our baseline, but a little slower than our NumPy example. Good news is its more memory efficient.

In [ ]:
%%time
#%%timeit
out_idx_nb_cpu, out_dist_nb_cpu = numba_cpu_solve(h_obs, h_ref)

## **Numba CUDA Kernel (Advanced)**

[Numba CUDA](https://numba.pydata.org/numba-doc/latest/cuda/index.html) provides us with an extremely pythonic interface to developing CUDA kernels without ever writing a single line of C/C++. It's pythonic syntax makes writing CUDA approachable to your typical data scientist. Numba CUDA exposes lower control to the developer including the use of shared memory, constant memory, atomics, warp level optimizations, and many others. Although not every element of CUDA is supported, developers are able to prototype highly performant algorithms very quickly. In many cases, these can be production ready techniques.

Numba CUDA implements the CUDA Array Interface, which means GPU data structures are interoperable between CuPy, cuDF, and Deep Learning Frameworks (Tensorflow, PyTorch). This is compelling from an end to end data processing perspecitve. Keeping the data on GPU for as long as possible let's developers avoid bottlenecks that creep in from unnecessary data copies across the PCIe bus.

Below, we implement several kernels to perform the nearest neighbor calculations. To demonstrate lower level CUDA support, our code performs a [sequential addressing tree based warp reduction](https://developer.nvidia.com/blog/using-cuda-warp-level-primitives/), leveraging efficient communcation between threads within the same warp.

Do not worry if we lost you here. Some of these are advanced concepts. The key takeaway is that more sophisticated CUDA programming is available to you without ever leaving the land of Python.

Learn more about the CUDA Programming Model [here](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)

Here are our hand tuned Numba JIT CUDA kernels that deploy warp level optimization to solve this problem.

In [ ]:
@cuda.jit(device=True, inline=True)
def _warp_min_reduce_idx_unrolled(val, idx):
    
    mask  = 0xffffffff    
        
    shfl_val = cuda.shfl_down_sync(
        mask, val, 16)
    
    shfl_idx = cuda.shfl_down_sync(
        mask, idx, 16)

    if val > shfl_val:
        val = shfl_val
        idx = shfl_idx
        
    shfl_val = cuda.shfl_down_sync(
        mask, val, 8)
    
    shfl_idx = cuda.shfl_down_sync(
        mask, idx, 8)

    if val > shfl_val:
        val = shfl_val
        idx = shfl_idx        
        
    shfl_val = cuda.shfl_down_sync(
        mask, val, 4)
    
    shfl_idx = cuda.shfl_down_sync(
        mask, idx, 4)

    if val > shfl_val:
        val = shfl_val
        idx = shfl_idx         
        
    shfl_val = cuda.shfl_down_sync(
        mask, val, 2)
    
    shfl_idx = cuda.shfl_down_sync(
        mask, idx, 2)

    if val > shfl_val:
        val = shfl_val
        idx = shfl_idx         
        
    shfl_val = cuda.shfl_down_sync(
        mask, val, 1)
    
    shfl_idx = cuda.shfl_down_sync(
        mask, idx, 1)

    if val > shfl_val:
        val = shfl_val
        idx = shfl_idx

    return val, idx

sig_block_get_nearest_brute = "void(float32[:,:], float32[:,:], uint32[:,:], float32[:,:])"
def _block_get_nearest_brute(coord1, coord2, block_idx, block_dist):
    
    """
    GPU accelerated pairwise distance comparisons in single
    precision.
    """    
    
    startx, starty = cuda.grid(2)
    stridex, stridey = cuda.gridsize(2)
    
    seed = float32(1e11)
        
    for i in range(starty, coord2.shape[0], stridey):    
        
        b_min_val = seed
        b_min_idx = uint32(0)
        coord2_i_0 = coord2[i,0]
        
        for j in range(startx, coord1.shape[0], stridex):

            coord1_j_0 = coord1[j,0]
        
            first_sin = sin(
                (coord2_i_0 - coord1_j_0) * float32(0.5))
            
            second_sin = sin(
                (coord2[i,1] - coord1[j, 1]) * float32(0.5))            

            local_val = float32(2.0) * asin(
                sqrt(
                    first_sin * first_sin + \
                    cos(coord1_j_0) * \
                    cos(coord2_i_0) * \
                    second_sin * second_sin)
            )            
            
            if local_val < b_min_val:
                b_min_val = local_val
                b_min_idx = j
                
                
        b_min_val, b_min_idx = \
            _warp_min_reduce_idx_unrolled(
            b_min_val, b_min_idx)

        if cuda.laneid == 0:
            block_dist[i, cuda.blockIdx.x] = b_min_val
            block_idx[i, cuda.blockIdx.x] = b_min_idx
            
sig_global_get_nearest_brute = "void(float32[:,:], uint32[:,:], float32[:], uint32[:])"
def _global_get_nearest_brute(block_dist, block_idx, out_dist, out_idx):        
        
    startx, starty = cuda.grid(2)
    stridex, stridey = cuda.gridsize(2)
    
    seed = float32(1e30)
    
    for i in range(starty, out_dist.shape[0], stridey):
        
        g_min_dist = seed
        g_min_idx = 0
        
        for j in range(startx, block_idx.shape[1], stridex):
            
            local_dist = block_dist[i, cuda.threadIdx.x]
            
            if local_dist < g_min_dist:
                g_min_dist = local_dist
                g_min_idx = block_idx[i, cuda.threadIdx.x]
        
        g_min_dist, g_min_idx = \
            _warp_min_reduce_idx_unrolled(
            g_min_dist, g_min_idx)
        
        if cuda.laneid == 0:
            out_dist[i] = g_min_dist
            out_idx[i] = g_min_idx

Create our JIT kernels

In [ ]:
block_min_reduce = cuda.jit(
    sig_block_get_nearest_brute,        
    fastmath=True)(_block_get_nearest_brute)

global_min_reduce = cuda.jit(
    sig_global_get_nearest_brute,        
    fastmath=True)(_global_get_nearest_brute)

### **Call our CUDA Kernel**

The function below solves our nearest neighbor problem in two steps. 

First, generate an intermediate solution by performing block level warp reductions by finding the closest 32 points for each data observation. The number 32 was selected to be equal to the size of a warp.

We demonstrate interoperability with CuPy by passing in CuPy device arrays to our kernel.

```block_min_reduce[bpg, tpb](d_ref, d_obs, d_block_idx, d_block_dist)```

Our first launch configuration:

* 2D grid of 3456 (a multiple of the # of GPU SMs) blocks -- 32x108
* Blocks contain 2D arrangement of 512 (a multiple of 32) threads -- 32x16
* Total of 1,769,472 threads

We make a second kernel call to find our global solution. Since our previous kernel implements block level parallelism, a second kernel call helps us with a global synchronization. In this phase, we compute a global solution to our nearest neighbor problem.

```global_min_reduce[bpg, tpb](d_block_dist, d_block_idx, d_out_dist, d_out_idx)```

Our second launch configuration:

* 1D grid of 108*20 blocks (multiple of the # of SMs)
* Blocks contain 2D arrangement of 512 (a multiple of 32) -- 32x16
* Total of 1,105,920 threads

We make sure the CPU waits for processing to complete before continuing by performing a global synchronization with ```cuda.synchronize()```

We achieve our final solution!

In [ ]:
def numba_cuda_solve(d_obs, d_ref):
    
    d_out_idx = cuda.device_array((d_obs.shape[0],), dtype=np.uint32)
    d_out_dist = cuda.device_array((d_obs.shape[0],), dtype=np.float32)     
    
    d_block_idx = cuda.device_array(
        (d_out_idx.shape[0], 32), 
        dtype=np.uint32)

    d_block_dist = cuda.device_array(
        (d_out_idx.shape[0], 32), 
        dtype=np.float32)           

    bpg = 32, 128
    tpb = 32, 16

    block_min_reduce[bpg, tpb](
        d_ref, 
        d_obs, 
        d_block_idx,
        d_block_dist)   

    bpg = (1, 128*20)
    tpb = (32, 16)        

    global_min_reduce[bpg, tpb](
        d_block_dist, 
        d_block_idx, 
        d_out_dist, 
        d_out_idx)   
        
    cuda.synchronize()
    
    return d_out_idx, d_out_dist

Verify our Numba CUDA kernels are producing the correct results.

In [ ]:
out_idx_nb_gpu_val, out_dist_nb_gpu_val = \
    numba_cuda_solve(d_obs_val, d_ref_val)

print("Accuracy - Numba CUDA Single GPU:", 
      check_accuracy(
          d_obs_val, d_ref_val,
          out_idx_nb_gpu_val, out_dist_nb_gpu_val)
     )

This Numba CUDA kernels complete on the demo system in ~10.8ms -- more than 200,000x faster than our baseline, and 1,129.6x faster than our most performant CPU option. In this scenario, Numba CUDA required more developer effort but provided us a framework to squeeze additional performance out of our system.

In [ ]:
%%timeit
out_idx_nb_cuda, out_dist_nb_cuda = numba_cuda_solve(d_obs, d_ref)

## **Please restart the kernel**